Tamrin1

In [4]:
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl


Class for fuzzification of errors and error deviations and calculate the PID controller

In [8]:
import numpy as np
import skfuzzy as fuzz
import skfuzzy.control as ctrl
# Threshold
def Fuzzifier_obstacle_volume():
     v= ctrl.Antecedent(np.arange(0, 0.7, 0.1))
     threshold=ctrl.Consequent(np.arange(0, 2, 0.1), f'_velocityob')
     v['S'] = fuzz.trimf(v.universe, [0, 0,0.1])       
     v['M'] = fuzz.trimf(v.universe, [0.05, 0.2, 0.35])      
     v['B'] = fuzz.trimf(v.universe, [0.35,0.5 ,0.7])
     threshold['S'] = fuzz.trimf(threshold.universe, [0, 0,0.5])       
     threshold['M'] = fuzz.trimf(threshold.universe, [0.5, 0.7, 1])      
     threshold['B'] = fuzz.trimf(threshold.universe, [1,1.5 ,2])
     rules=[
            ctrl.Rule(v['NB'] , threshold['B']),
            ctrl.Rule(v['NB'] , threshold['M']),
            ctrl.Rule(v['NB'], threshold['S'])]
     ctrl_vrules = ctrl.ControlSystem(rules)
     sim_vrules = ctrl.ControlSystemSimulation(ctrl_vrules)
     sim_vrules.input = v
     sim_vrules.compute()
     return sim_vrules.output

# Define the JointFuzzyController class
class JointFuzzyController:
    def __init__(self, joint_name):
        self.e = ctrl.Antecedent(np.arange(-1.5, 1.5, 0.1), f'{joint_name}_e')
        self.ec = ctrl.Antecedent(np.arange(-1.5, 1.5, 0.1), f'{joint_name}_ec')
        self.myvelocity = ctrl.Antecedent(np.arange(0, 2, 0.1), f'{joint_name}_myvelocity')
        self.distance_obstacle = ctrl.Antecedent(np.arange(0, 21, 0.1), f'{joint_name}_distance_obstacle')
        self.distance_target = ctrl.Antecedent(np.arange(0, 21, 0.1), f'{joint_name}_distance_target')
        self.threshold = 0
        self.velocity = ctrl.Consequent(np.arange(0, 2, 0.1), f'{joint_name}_velocity')
        self.velocityob = ctrl.Consequent(np.arange(0, 2, 0.1), f'{joint_name}_velocityob')
        self._init_membership_functions_ob(self.threshold)
        self._init_membership_functions()
        self._init_rules_velocity()
        self._init_rules_ob_velocity()

        # Initialize the control systems
        self.ctrl_vrules = ctrl.ControlSystem(self.vrules)
        self.sim_vrules = ctrl.ControlSystemSimulation(self.ctrl_vrules)
        
        self.ctrl_obvrules = ctrl.ControlSystem(self.obvrules)
        self.sim_obvrules = ctrl.ControlSystemSimulation(self.ctrl_obvrules)
    def _init_membership_functions_ob(self,x):
        # Membership functions for distance with the obstacle       
        self.distance_obstacle['VN'] = fuzz.trimf(self.distance_obstacle.universe, [0, 0+x/2, 3+x])       # Very Near
        self.distance_obstacle['N'] = fuzz.trimf(self.distance_obstacle.universe, [2, 3+x/2, 5+x])       # Near
        self.distance_obstacle['F'] = fuzz.trapmf(self.distance_obstacle.universe, [4, 5, 10, 16])      # Far
        self.distance_obstacle['VF'] = fuzz.trapmf(self.distance_obstacle.universe, [15, 16, 20, 21])    # Very Far

        # Membership functions for distance with the target
        self.distance_target['VN'] = fuzz.trimf(self.distance_target.universe, [0, 0, 3])       # Very Near
        self.distance_target['N'] = fuzz.trimf(self.distance_target.universe, [2, 3, 5])       # Near
        self.distance_target['F'] = fuzz.trapmf(self.distance_target.universe, [4, 5, 10, 16])      # Far
        self.distance_target['VF'] = fuzz.trapmf(self.distance_target.universe, [15, 16, 20, 21])    # Very Far
    def _init_membership_functions(self):
        # Membership functions for error (e)
        self.e['NB'] = fuzz.trimf(self.e.universe, [-1.5, -1.5, -0.75])
        self.e['NS'] = fuzz.trimf(self.e.universe, [-1.5, 0, 0.75])
        self.e['ZO'] = fuzz.trimf(self.e.universe, [-0.75, 0, 0.75])
        self.e['PS'] = fuzz.trimf(self.e.universe, [0, 0.75, 1.5])
        self.e['PB'] = fuzz.trimf(self.e.universe, [0.75, 1.5, 1.5])

        # Membership functions for error change (ec)
        self.ec['NB'] = fuzz.trimf(self.ec.universe, [-1.5, -1.5, -0.75])
        self.ec['NS'] = fuzz.trimf(self.ec.universe, [-1.5, 0, 0.75])
        self.ec['ZO'] = fuzz.trimf(self.ec.universe, [-0.75, 0, 0.75])
        self.ec['PS'] = fuzz.trimf(self.ec.universe, [0, 0.75, 1.5])
        self.ec['PB'] = fuzz.trimf(self.ec.universe, [0.75, 1.5, 1.5])

        # Membership functions for velocity
        self.velocity['Slow'] = fuzz.trimf(self.velocity.universe, [0, 0, 1])
        self.velocity['Medium'] = fuzz.trimf(self.velocity.universe, [0.5, 1, 1.5])
        self.velocity['Fast'] = fuzz.trimf(self.velocity.universe, [1, 2, 2])
        #Define velocity with obstacle
        # Define fuzzy membership functions for velocity
        self.velocityob['VS'] = fuzz.trimf(self.velocityob.universe, [0, 0, 0.5])     # Very Slow
        self.velocityob['MS'] = fuzz.trimf(self.velocityob.universe, [0, 0.5, 1])      # Middle Slow
        self.velocityob['S'] = fuzz.trimf(self.velocityob.universe, [0.5, 1, 1.5])     # Slow
        self.velocityob['M'] = fuzz.trimf(self.velocityob.universe, [1, 1.5, 2])       # Middle
        self.velocityob['F'] = fuzz.trimf(self.velocityob.universe, [1.5, 2, 2])       # Fast
        self.velocityob['MF'] = fuzz.trimf(self.velocityob.universe, [1.75, 2, 2])     # Middle Fast
        self.velocityob['VF'] = fuzz.trimf(self.velocityob.universe, [2, 2, 2])        # Very Fast 

        # my velocity
        self.myvelocity['VS'] = fuzz.trimf(self.myvelocity.universe, [0, 0, 0.5])     # Very Slow
        self.myvelocity['MS'] = fuzz.trimf(self.myvelocity.universe, [0, 0.5, 1])      # Middle Slow
        self.myvelocity['S'] = fuzz.trimf(self.myvelocity.universe, [0.5, 1, 1.5])     # Slow
        self.myvelocity['M'] = fuzz.trimf(self.myvelocity.universe, [1, 1.5, 2])       # Middle
        self.myvelocity['F'] = fuzz.trimf(self.myvelocity.universe, [1.5, 2, 2])       # Fast
        self.myvelocity['MF'] = fuzz.trimf(self.myvelocity.universe, [1.75, 2, 2])     # Middle Fast
        self.myvelocity['VF'] = fuzz.trimf(self.myvelocity.universe, [2, 2, 2])        # Very Fast

    def _init_rules_velocity(self):
        # Define rules for velocity control
        self.vrules = [
            ctrl.Rule(self.e['NB'] & self.ec['NB'], self.velocity['Slow']),
            ctrl.Rule(self.e['NB'] & self.ec['NS'], self.velocity['Slow']),
            ctrl.Rule(self.e['NB'] & self.ec['ZO'], self.velocity['Medium']),
            ctrl.Rule(self.e['NB'] & self.ec['PS'], self.velocity['Medium']),
            ctrl.Rule(self.e['NB'] & self.ec['PB'], self.velocity['Fast']),

            ctrl.Rule(self.e['ZO'] & self.ec['NB'], self.velocity['Medium']),
            ctrl.Rule(self.e['ZO'] & self.ec['NS'], self.velocity['Medium']),
            ctrl.Rule(self.e['ZO'] & self.ec['ZO'], self.velocity['Medium']),
            ctrl.Rule(self.e['ZO'] & self.ec['PS'], self.velocity['Medium']),
            ctrl.Rule(self.e['ZO'] & self.ec['PB'], self.velocity['Fast']),

            ctrl.Rule(self.e['PB'] & self.ec['NB'], self.velocity['Medium']),
            ctrl.Rule(self.e['PB'] & self.ec['NS'], self.velocity['Medium']),
            ctrl.Rule(self.e['PB'] & self.ec['ZO'], self.velocity['Fast']),
            ctrl.Rule(self.e['PB'] & self.ec['PS'], self.velocity['Fast']),
            ctrl.Rule(self.e['PB'] & self.ec['PB'], self.velocity['Fast']),
        ]
    def _init_rules_ob_velocity(self):
        self.obvrules = [
            ctrl.Rule(self.distance_target['VN'] & self.distance_obstacle['VN'] & self.myvelocity['VS'], self.velocityob['VS']),
            ctrl.Rule(self.distance_target['VN'] & self.distance_obstacle['VN'] & self.myvelocity['MS'], self.velocityob['VS']),
            ctrl.Rule(self.distance_target['VN'] & self.distance_obstacle['VN'] & self.myvelocity['S'], self.velocityob['VS']),
            ctrl.Rule(self.distance_target['VN'] & self.distance_obstacle['VN'] & self.myvelocity['M'], self.velocityob['MS']),
            ctrl.Rule(self.distance_target['VN'] & self.distance_obstacle['VN'] & self.myvelocity['F'], self.velocityob['M']),
            ctrl.Rule(self.distance_target['VN'] & self.distance_obstacle['VN'] & self.myvelocity['MF'], self.velocityob['M']),
            ctrl.Rule(self.distance_target['VN'] & self.distance_obstacle['VN'] & self.myvelocity['VF'], self.velocityob['F']),
            ctrl.Rule(self.distance_target['VN'] & self.distance_obstacle['N'] & self.myvelocity['VS'], self.velocityob['VS']),
            ctrl.Rule(self.distance_target['VN'] & self.distance_obstacle['N'] & self.myvelocity['MS'], self.velocityob['VS']),
            ctrl.Rule(self.distance_target['VN'] & self.distance_obstacle['N'] & self.myvelocity['S'], self.velocityob['MS']),
            ctrl.Rule(self.distance_target['VN'] & self.distance_obstacle['N'] & self.myvelocity['M'], self.velocityob['S']),
            ctrl.Rule(self.distance_target['VN'] & self.distance_obstacle['N'] & self.myvelocity['F'], self.velocityob['M']),
            ctrl.Rule(self.distance_target['VN'] & self.distance_obstacle['N'] & self.myvelocity['MF'], self.velocityob['F']),
            ctrl.Rule(self.distance_target['VN'] & self.distance_obstacle['N'] & self.myvelocity['VF'], self.velocityob['MF']),
            ctrl.Rule(self.distance_target['N'] & self.distance_obstacle['N'] & self.myvelocity['VS'], self.velocityob['VS']),
            ctrl.Rule(self.distance_target['N'] & self.distance_obstacle['N'] & self.myvelocity['MS'], self.velocityob['VS']),
            ctrl.Rule(self.distance_target['N'] & self.distance_obstacle['N'] & self.myvelocity['S'], self.velocityob['MS']),
            ctrl.Rule(self.distance_target['N'] & self.distance_obstacle['N'] & self.myvelocity['M'], self.velocityob['S']),
            ctrl.Rule(self.distance_target['N'] & self.distance_obstacle['N'] & self.myvelocity['F'], self.velocityob['M']),
            ctrl.Rule(self.distance_target['N'] & self.distance_obstacle['N'] & self.myvelocity['MF'], self.velocityob['F']),
            ctrl.Rule(self.distance_target['N'] & self.distance_obstacle['N'] & self.myvelocity['VF'], self.velocityob['MF']),
            ctrl.Rule(self.distance_target['N'] & self.distance_obstacle['F'] & self.myvelocity['VS'], self.velocityob['VS']),
            ctrl.Rule(self.distance_target['N'] & self.distance_obstacle['F'] & self.myvelocity['MS'], self.velocityob['VS']),
            ctrl.Rule(self.distance_target['N'] & self.distance_obstacle['F'] & self.myvelocity['S'], self.velocityob['MS']),
            ctrl.Rule(self.distance_target['N'] & self.distance_obstacle['F'] & self.myvelocity['M'], self.velocityob['S']),
            ctrl.Rule(self.distance_target['N'] & self.distance_obstacle['F'] & self.myvelocity['F'], self.velocityob['M']),
            ctrl.Rule(self.distance_target['N'] & self.distance_obstacle['F'] & self.myvelocity['MF'], self.velocityob['F']),
            ctrl.Rule(self.distance_target['N'] & self.distance_obstacle['F'] & self.myvelocity['VF'], self.velocityob['MF']),
        ]


    def determine_best_distance_category(self, distance_value):
    # Calculate the membership values for each distance category
        distance_membership_vf = fuzz.interp_membership(self.distance_obstacle.universe, self.distance_obstacle['VF'].mf, distance_value)
        distance_membership_f = fuzz.interp_membership(self.distance_obstacle.universe, self.distance_obstacle['F'].mf, distance_value)
        distance_membership_n = fuzz.interp_membership(self.distance_obstacle.universe, self.distance_obstacle['N'].mf, distance_value)
        distance_membership_vn = fuzz.interp_membership(self.distance_obstacle.universe, self.distance_obstacle['VN'].mf, distance_value)

    # Create a dictionary to store membership values
        membership_values = {
            'VF': distance_membership_vf,
            'F': distance_membership_f,
            'N': distance_membership_n,
            'VN': distance_membership_vn
        }

    # Determine the category with the highest membership value
        best_category = max(membership_values, key=membership_values.get)

        return best_category

    def compute_velocity_ob(self, error, error_change, myvelocity, distance_target_v, distance_obstacle_value,threshold):
    # Determine the best distance categories for obstacle and target
        best_distance_obstacle = self.determine_best_distance_category(distance_obstacle_value)
        best_distance_target = self.determine_best_distance_category(distance_target_v)
        self.threshold=threshold
    # Check conditions to determine which rules to use or return 20 to change the path
        if best_distance_obstacle == 'VF':
            self.sim_vrules.input[f'{self.e.label}'] = error
            self.sim_vrules.input[f'{self.ec.label}'] = error_change
            self.sim_vrules.compute()
            return self.sim_vrules.output[f'{self.velocity.label}']
        elif best_distance_obstacle == 'F' and (best_distance_target != 'N'):
            self.sim_vrules.input[f'{self.e.label}'] = error
            self.sim_vrules.input[f'{self.ec.label}'] = error_change
            self.sim_vrules.compute()
            return self.sim_vrules.output[f'{self.velocity.label}']
        elif best_distance_obstacle == 'N' and (best_distance_target == 'F' or best_distance_target=='VF'):
            return 20
        elif best_distance_obstacle == 'VN' and (best_distance_obstacle != 'VN'):
            return 20
        else:
            self.sim_obvrules.input[f'{self.distance_target.label}'] = distance_target_v
            self.sim_obvrules.input[f'{self.distance_obstacle.label}'] = distance_obstacle_value
            self.sim_obvrules.input[f'{self.myvelocity.label}'] = myvelocity
            self.sim_obvrules.compute()
            return self.sim_obvrules.output[f'{self.velocityob.label}']

    def compute_velocity(self, error, error_change):
        self.sim_vrules.input[f'{self.e.label}'] = error
        self.sim_vrules.input[f'{self.ec.label}'] = error_change
        self.sim_vrules.compute()
        return self.sim_vrules.output[f'{self.velocity.label}']


